In [1]:
import os
from mltk.utils.path import create_tempdir


# This contains the path to the pre-trained model in ONNX model format
# For this tutorial, we use the one downloaded from above
# Update this path to point to your specific model if necessary
ONNX_MODEL_PATH = 'C:/Users/satya/OneDrive/Desktop/Vehicle Damage Detection System/best.onnx'


WORKING_DIR = create_tempdir('C:/Users/satya/OneDrive/Desktop/Vehicle Damage Detection System/model_onnx_to_tflite')



assert os.path.exists(ONNX_MODEL_PATH), f'The provided ONNX_MODEL_PATH does not exist at: {ONNX_MODEL_PATH}'
os.makedirs(WORKING_DIR, exist_ok=True)


# Use the filename for the model's name
MODEL_NAME = os.path.basename(ONNX_MODEL_PATH)[:-len('.onnx')]


print(f'ONNX_MODEL_PATH = {ONNX_MODEL_PATH}')
print(f'MODEL_NAME = {MODEL_NAME}')
print(f'WORKING_DIR = {WORKING_DIR}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\satya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\satya\AppData\Local\Temp\ipykernel_16272\2452898687.py", line 2, in <module>
    from mltk.utils.path import create_tempdir
ModuleNotFoundError: No module named 'mltk.utils'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\satya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\satya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\L

In [5]:
pip install visualization


     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.6 kB 1.4 MB/s eta 0:00:01
     ------------------------- ------------ 41.0/60.6 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 461.8 kB/s eta 0:00:00
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
     ---------------------------------------- 0.0/109.2 kB ? eta -:--:--
     --- ------------------------------------ 10.2/109.2 kB ? eta -:--:--
     ------------- ----------------------- 41.0/109.2 kB 653.6 kB/s eta 0:00:01
     -------------------- ---------------- 61.4/109.2 kB 648.1 kB/s eta 0:00:01
     ------------------------------- ----- 92.2/109.2 kB 655.4 kB/s eta 0:00:01
     ------------------------------------ 109.2/109.2 kB 630.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cycler-0.12.1-py3-none-a

In [2]:
import numpy as np

# This is the class label order specified by the dataset
# y_test contains a list of integers that correspond to the indices in this class_labels list
       
class_labels = ['damaged door', 'damaged window', 'damaged headlight', 'damaged mirror', 'dent', 'damaged hood', 'damaged bumper', 'damaged wind shield']

In [7]:
import os
import numpy as np
from PIL import Image

def load_data(data_dir):
    """Loads custom dataset and preprocesses it to match CIFAR-10.
    Assumes the images are stored in the directory structure
    `data_dir/class_name/image_name.jpg`.
    Returns:
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    classes = sorted(os.listdir(data_dir))

    # Count the number of samples in the training and test sets
    num_train_samples = 0
    num_test_samples = 0
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        num_images = len(os.listdir(class_dir))
        num_train = int(num_images * 0.8)
        num_test = num_images - num_train
        num_train_samples += num_train
        num_test_samples += num_test

    # Initialize the arrays to hold the data
    x_train = np.zeros((num_train_samples, 32, 32, 3), dtype='float32')
    y_train = np.zeros((num_train_samples,), dtype='uint8')
    x_test = np.zeros((num_test_samples, 32, 32, 3), dtype='float32')
    y_test = np.zeros((num_test_samples,), dtype='uint8')

    # Load the data from each class
    train_index = 0
    test_index = 0
    for class_index, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        image_names = os.listdir(class_dir)
        np.random.shuffle(image_names)
        num_images = len(image_names)
        num_train = int(num_images * 0.8)
        num_test = num_images - num_train
        for i in range(num_train):
            image_name = image_names[i]
            image_path = os.path.join(class_dir, image_name)
            img = Image.open(image_path)
            img = img.resize((32, 32))
            x_train[train_index] = np.array(img, dtype='float32') / 255.0
            y_train[train_index] = class_index
            train_index += 1
        for i in range(num_test):
            image_name = image_names[num_train + i]
            image_path = os.path.join(class_dir, image_name)
            img = Image.open(image_path)
            img = img.resize((32, 32))
            x_test[test_index] = np.array(img, dtype='float32') / 255.0
            y_test[test_index] = class_index
            test_index += 1

    return (x_train, y_train), (x_test, y_test)


In [12]:
data_dir='test_images'
(_, _), (x_test, y_test) = load_data(data_dir)


In [13]:
x_test = x_test.astype('float32')
# Scale the samples by 255 since that's the preprocessing
# used during model training
x_test = x_test/255.


In [14]:
print(f'x_test.shape = {x_test.shape}')
print(f'y_test.shape = {y_test.shape}')

x_test.shape = (97, 32, 32, 3)
y_test.shape = (97,)


In [24]:
mapped_class_labels = ['damaged door', 'damaged window', 'damaged headlight', 'damaged mirror', 'dent', 'damaged hood', 'damaged bumper', 'damaged wind shield']
CLASS_ID_MAPPING = {
0: 0, 
1: 1, 
2: 2, 
3: 3 ,
4: 4,
5: 5,
6: 6,
7: 7
}



In [25]:
y_test = y_test.reshape(-1)

In [26]:
for i, y in enumerate(y_test):
    y_test[i] = CLASS_ID_MAPPING[y]

In [30]:
import tensorflow as tf

# Create a tensor with 1344 elements
tensor_1344 = tf.ones((64, 21, 5))
print("Original tensor shape:", tensor_1344.shape)

# Reshape tensor with 1344 elements to shape [1,4,16,21,5]
new_shape = (1, 4, 16, 21, 5)

# Add missing dimensions
missing_dims = len(new_shape) - len(tensor_1344.shape)
if missing_dims > 0:
    tensor_1344 = tf.reshape(tensor_1344, tuple(tensor_1344.shape) + (1,) * missing_dims)

# Reshape tensor to new shape
reshaped_tensor = tf.reshape(tensor_1344, new_shape)
print("Reshaped tensor shape:", reshaped_tensor.shape)


Original tensor shape: (64, 21, 5)
Reshaped tensor shape: (1, 4, 16, 21, 5)


In [31]:
import onnx
from onnx_tf.backend import prepare

# Load the ONNX model
onnx_model = onnx.load(ONNX_MODEL_PATH)
tf_rep = prepare(onnx_model)

n_samples = min(len(x_test), 1000) # Let's evaluate up to 1000 samples
n_classes = len(mapped_class_labels)

# Allocate an array to hold the model predictions
y_pred = np.empty((n_samples, n_classes), dtype=np.float32)
print(f'y_pred.shape = {y_pred.shape}')

# The dataset uses the format: NHWC (i.e. channels last)
# However, the ONNX model expects NCHW (i.e. channels first)
# So transpose the x_test data to be in NCHW format
x_test_channels_first = x_test.transpose(0, 3, 1, 2)

# Iterate through each test sample
print(f'Generating model predictions for each test sample using {ONNX_MODEL_PATH}')
print('Be patient, this may take awhile ...')
for i, x in enumerate(x_test_channels_first[:n_samples]):
    # Add the N dimension to the individual sample
    # e.g. CHW -> NCHW
    x = np.expand_dims(x, 0)
    # Run inference on the sample
    outputs = tf_rep.run(x)
    # Save the model prediction
    y_pred[i] = outputs[0]

print('done')

y_pred.shape = (97, 8)
Generating model predictions for each test sample using /home/surya/Desktop/damaged_car_parts_detection/runs/detect/train3/weights/best.onnx
Be patient, this may take awhile ...


ValueError: in user code:

    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/handlers/handler.py", line 59, in handle  *
        return ver_handle(node, **kwargs)
    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/handlers/backend/reshape.py", line 64, in version_14  *
        return cls._common(node, **kwargs)
    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/handlers/backend/reshape.py", line 44, in _common  *
        cls.make_tensor_from_onnx_node(node,
    File "/home/surya/.local/lib/python3.10/site-packages/onnx_tf/handlers/backend_handler.py", line 157, i

In [32]:
import cv2

# Load Model
net = cv2.dnn.readNet('/home/surya/Desktop/damaged_car_parts_detection/runs/detect/train3/weights/best.onnx')

[ERROR:0@1.317] global onnx_importer.cpp:1051 handleNode DNN/ONNX: ERROR during processing node with 2 inputs and 1 outputs: [Reshape]:(onnx_node!/model.22/dfl/Reshape) from domain='ai.onnx'


error: OpenCV(4.7.0) /io/opencv/modules/dnn/src/onnx/onnx_importer.cpp:1073: error: (-2:Unspecified error) in function 'handleNode'
> Node [Reshape@ai.onnx]:(onnx_node!/model.22/dfl/Reshape) parse error: OpenCV(4.7.0) /io/opencv/modules/dnn/src/layers/reshape_layer.cpp:109: error: (-215:Assertion failed) total(srcShape, srcRange.start, srcRange.end) == maskTotal in function 'computeShapeByReshapeMask'
> 